## 학습정리

### 16장. 로지스틱 회귀 분석
* 종속 변수가 범주형 데이터
* 일종의 분류 기법

####  16.1 문제
* ex. 유료 계정으로 전환할지 예측
    * 선형 회귀 분석 : 유료 계정 등록 여부 = $\beta_0 + \beta_1 경력 + \beta_2소득 + \epsilon$
        * 발생하는 몇가지 문제점 
            1. 결과값이 아주 큰 양수값 또는 음수값이므로 0과1로 등록여부가 결정되는 예측값을 해석하기 힘들다
            2. beta에 관한 추정이 편향됨 : 실제 y의 최댓값은 1, 아주 큰 예측값은 큰 음의 오류를 발생시킴

####  16.2 로지스틱 함수
* 로지스틱 회귀 분석 : 로지스틱 함수(logistic function)를 사용
    * 로지스틱 함수는 입력값이 양의 방향으로 커질수록 출력값이 1에 가까워짐
    * $y_i = f(x_i\beta)+\epsilon$
    * 선형 회귀 분석에서의 가능도(likelihood) : 오차 제곱 합을 최소화 하는 베타가 가능도도 최대화 함
    * 로지스틱 회귀 분석에서의 가능도 : 오차 제곱합과 별개
        * 경사 하강법(gradient descent)으로 직접 최대화
        * $\beta$가 주어졌을 때 각 $y_i$ 
            * $f(x_i\beta)의 확률로 1$
            * $1-f(x_i\beta)의 확률로 0$
* $y_i$의 확률 밀도 함수(probability density function)
    * $p(y_i|x_i,\beta) = f(x_i\beta)^{y_i}(1-f(x_i\beta))^{1-y_i}$
        * $y_i$가 0이면 : $1-f(x_i\beta)$
        * $y_i$가 1이면 : $f(x_i\beta)$
    * 로그 가능도(log likelihood)를 최대화
        * $logL(\beta|x_i,y_i) = y_ilogf(x_i\beta)+(1-y_i)log(1-f(x_i\beta))$
        * 로그 함수 : 단조 증가함수, 로그 가능도를 최대화 하는 beta는 가능도 또한 최대화 (역(최소화)도 성립)
            * 경사 하강법은 함수를 최소화 할 때 사용하므로 로그 가능도의 부호를 바꾼 네거티브(negative)로 그 가능도를 사용
        * 데이터 포인트 끼리 서로 독립일 때 데이터 전체의 가능도 = 개별 데이터 포인트의 가능도의 단순 곱
            * 데이터 전체에 대한 로그 가능도 = 개별 데이터 포인트의 가능도의 단순 합 
            * 미분해서 그래디언트를 구할 수 있음
    
####  16.3 모델 적용하기
* 데이터를 학습 데이터와 테스트 데이터로 분할, 경사 하강법(batch gradient descent)을 적용
* 결과를 통해서 다른 조건이 동일할때 한가지 특성의 영향은 파악할 수 있다 

####  16.4 적합성(Goodness of fit)
* 평가용으로 따로 빼 놓았던 데이터를 사용
    * 예측 값이 0.5를 초과할 때마다 사용자가 유료 계정으로 등록한다고 예측
* 정밀도(precision) : TP / (TP+FP) - 유료계정이라고 예측했는데 유료계정 이용자일 확률 
* 재현율(recall) : TP / (TP+FN) - 유료 계정 이용자를 유료 계정이용자라고 예측할 확률

####  16.5 서포트 벡터 머신
* dot(beta_hat, x_i) = 0인 부분이 두 클래스의 경계면
* 초평면(hyperplane) : 전체 파라미터 공간을 '유료계정'과 '유료 계정이 아닌' 두개의 부분 공간으로 나누는 경계면
    * 가능도를 최대화하는 결과의 부산물로 얻을 수 있음
* 애초에 초평면을 찾는 것을 목적으로 하는 것도 존재 : support vector machine(SVM)     
    * 각 클래스 안의 초평면과 가장 가까운 점의 거리를 최대화 하는 방식
    * 데이터를 분류할 수 있는 초평면이 아예 존재하지 않을 수도 있음
* 커널 트릭(kernel trick) : 데이터가 선형적으로 구분될 수 있는 조금 더 차원이 높은 공간으로 데이터를 변환하여 보냄
    * 데이터 포인트를 더 높은 차원의 공간으로 직접 매핑하기 보다 '커널 함수'로 더 높은 차원에서의 내적을 계산하고 그것으로 초평면을 찾음
    
####  16.6 더 공부해 보고 싶다면
* scikit-learn : 로지스틱 회귀 분석, 서포트 벡터 머신을 위한 모델 제공
* scikit-learn - LIBSVM : 서포트 벡터 머신 모듈이 사용하는 구현체 

## code

In [1]:
# 로지스틱 함수
def logistic(x : float) -> float :
    return 1.0 / (1+math.exp(-x))

# 로지스틱 함수 미분 
def logistic_prime(x: float) -> float :
    y = logistic(x)
    return y *(1-y)





In [2]:
# 네거티브 로그 가능도(경사 하강법은 최소화하는 함수이므로 가능도의 부호를 바꿔서 찾음)
import math
from typing import List
Vector = List[float]
import numpy as np

def _negative_log_likelihood(x : Vector, y: float, beta : Vector) -> float :
    """데이터 포인트의 네거티브 로그 가능도"""
    if y == 1:
        return -math.log(logistic(np.dot(x,beta))) # f
    else :
        return -math.log(1 - logistic(dot(x,beta))) # 1-f
    
    
# 로그 가능도의 단순 합
def negative_log_likelihood(xs : List[Vector],
                           ys : List[float],
                           beta : Vector) -> float :
    return sum(_negative_log_likelihood(x, y, beta) for x, y in zip(xs, ys))

# 미적분 
# vector_sum : np.sum(x , axis = 0)

def _negative_log_partial_j(x: Vector, y: float, beta: Vector, j: int) -> float :
    """데이터 포인트 j번째 편미분, 여기서 i는 데이터 포인트의 인덱스를 의미"""
    return -(y - logistic(dot(x, beta))) * x[j]

def _negative_log_gradient(x: Vector, y: float, beta: Vector) -> Vector :
    """데이터 포인트의 그래디언트 """
    return [_negative_log_partial_j(x, y, beta, j) for j in range(len(beta))]

def negative_log_gradient(xs : List[Vector], ys : List[float], beta : Vector) -> Vector :
    return vector_sum([_negative_log_gradient(x, y, beta) for x,y in zip(xs, ys)])

In [3]:
# 모델 적용
import random
from sklearn.model_selection import train_test_split
import tqdm

random.seed(0)
x_train, x_test, y_train, y_test = train_test_split(recaled_xs, ys, 0.33) # machine_learning 

learning_rate = 0.01

# 임의의 시작점
beta = [random.random() for _ in range(3)]

with tqdm.trange(5000) as t :
    for epoch in t :
        gradient = negative_log_gradient(x_train,y_train, beta)
        beta = gradient_step(beta, gradient, -learning_rate)
        loss = negative_log_likelihood(x_train, y_train, beta)
        t.set_description(f"loss: {loss:.3f} beta: {beta}")

# beta = [-2.0, 4.7, -4.5]

NameError: name 'recaled_xs' is not defined

In [ ]:
# rescale된 데이터를 다시 변환
from scratch.working_with_data import scale

means, stdevs = scale(xs)
beta_unscaled = [(beta[0] - beta[1]*means[1] / stedvs[1] - beta[2]*means[2] / stedvs[2]),
                beta[1] / stedvs[1],
                beta[2] / stedvs[2]]

# [8.9, 1.6, -0.000288]


In [ ]:
# 적합성
true_positives = false_positives = true_negatives = false_negatives = 0

for x_i, y_i in zip(x_test, y_test) :
    prediction = logistic(dot(beta, x_i))
    
    if y_i == 1 and prediction >= 0.5 :
        true_positives += 1
    elif y_i == 1:
        false_negatives += 1
    elif prediction >= 0.5 :
        false_positives += 1
    else :
        true_negatives += 1

precision = true_positives / (true_positives + false_positives) # 정밀도
recall = true_positives / (true_positives + false_negatives) # 재현율


In [ ]:
# 예측값 시각화
predictions = [logistic(np.dot(beta, x_i)) for x_i in x_test]
plt.scatter(predictions, y_test, marker = '+')
plt.xlabel("predicted probability")
plt.ylabel("actual outcome")
plt.title("Logistic Regression Predicted vs. Actual")
plt.show()
